In [1]:
using Plots # for plotting
using Base.Threads # for parallelization
using StaticArrays # somehow needed to use multiple variables in DifferentialEquations.jl

using Plots 
using LaTeXStrings, Colors
using Plots.PlotMeasures
using LinearAlgebra

using Random, Distributions

using FFTW # discrete Fourier transform

using JLD2 # for file saving
using DataFrames, Statistics

In [2]:
phicolor = colorant"#c51b8a";
chicolor = colorant"#00FFFF";

# import and process data

In [3]:
function load_results_as_dataframe(filename::AbstractString)
    results = DataFrame(
        stableRandomSeed = Int[],
        amp = Float64[],
        ratioPhi = Float64[],
        ratioChi = Float64[]
    )

    jldopen(filename, "r") do file
        for case_id in keys(file)
            case = file[case_id]
            push!(results, (
                case[:stableRandomSeed],
                case[:amp],
                case[:ratioPhi],
                case[:ratioChi]
            ); promote=true)
        end
    end

    return results
end

load_results_as_dataframe (generic function with 1 method)

Load ++ ID dataset.

In [4]:
dir_path = "++/dat"
filename = joinpath(
    pwd(),
    dir_path,
    "results.jld2"
)

dfID = load_results_as_dataframe(filename)

# df = filter(row -> row.stableRandomSeed != 8597453 , df)

unique(dfID.stableRandomSeed)

1-element Vector{Int64}:
 0

In [5]:
dfID.ratio = (dfID.ratioPhi + dfID.ratioChi)/2;
dfID.ratioMax = max.(dfID.ratioPhi,dfID.ratioChi);
#println(df)

Load random ID datasets.

In [14]:
dir_path = "random_amplitude_random_width/dat"
filename = joinpath(
    pwd(),
    dir_path,
    "results1.jld2"
)

df1 = load_results_as_dataframe(filename)

dir_path = "random_amplitude_random_width/dat"
filename = joinpath(
    pwd(),
    dir_path,
    "results2.jld2"
)

df2 = load_results_as_dataframe(filename)

df = vcat(df1, df2)

# df = filter(row -> row.stableRandomSeed != 8597453 , df)

unique(df.stableRandomSeed)

50-element Vector{Int64}:
 9920130
 2167148
 4594825
 9289607
 9082945
 2789335
 4966556
 3579194
 3774167
 7887472
 3073115
 3422736
 1878673
       ⋮
 6321166
 3838545
 8240556
 4253005
 6239779
 8602818
 2527185
  764136
 6083695
 5956860
  734469
 5056809

In [15]:
df.ratio = (df.ratioPhi + df.ratioChi)/2;
df.ratioMax = max.(df.ratioPhi,df.ratioChi);
#println(df)

In [16]:
for seed in unique(df.stableRandomSeed)
    df_filtered = filter(row -> row.stableRandomSeed == seed , df)
    println("seed: ",seed, ", cases: ", length(df_filtered.amp))
end

seed: 9920130, cases: 80
seed: 2167148, cases: 80
seed: 4594825, cases: 80
seed: 9289607, cases: 80
seed: 9082945, cases: 80
seed: 2789335, cases: 80
seed: 4966556, cases: 80
seed: 3579194, cases: 80
seed: 3774167, cases: 79
seed: 7887472, cases: 79
seed: 3073115, cases: 80
seed: 3422736, cases: 80
seed: 1878673, cases: 80
seed: 2923292, cases: 80
seed: 4087734, cases: 80
seed: 4503507, cases: 80
seed: 1084538, cases: 80
seed: 7266251, cases: 79
seed: 4702800, cases: 79
seed: 6119173, cases: 79
seed: 6771503, cases: 80
seed: 3822863, cases: 80
seed: 4203244, cases: 80
seed: 5668746, cases: 80
seed: 6924766, cases: 79
seed: 303193, cases: 79
seed: 8780564, cases: 79
seed: 8877992, cases: 79
seed: 264962, cases: 79
seed: 245663, cases: 79
seed: 1801466, cases: 80
seed: 5836599, cases: 80
seed: 1966545, cases: 80
seed: 1304088, cases: 80
seed: 9853834, cases: 80
seed: 6680495, cases: 80
seed: 4838546, cases: 80
seed: 7175382, cases: 64
seed: 6321166, cases: 79
seed: 3838545, cases: 79
see

In [17]:
# Group by :amp
grouped = groupby(df, :amp)

# Compute means and stds for ratioPhi and ratioChi in one go
stats_df = combine(grouped, 
    :ratioPhi => mean => :mean_ratioPhi, 
    :ratioPhi => std  => :std_ratioPhi,
    :ratioChi => mean => :mean_ratioChi, 
    :ratioChi => std  => :std_ratioChi
);

# plot critical amplitude

In [18]:
frame = plot(
    layout=(1), size = (600,400), framestyle=:box, dpi = 400, 
    guidefont = "Computer Modern", tickfont = "Computer Modern",
    xguidefontsize = 16, yguidefontsize = 16, 
    legendfontsize = 16, titlefontsize = 16, tickfontsize = 16,
    legend=:topleft,
    title=L"$(N,\ell,m,n)=(3,6,2,2),\;\lambda_{mn}=1,\;\lambda_\ell=1$",
    xlabel=L"$\textrm{amplitude}\;\;\overline{A}$",
    #xscale=:log,
    #xticks = 0:1:10,
    #xrange = (1.,13),
    ylabel=L"$\textrm{extracted}\;\textrm{energy}\;\textrm{fraction}$",
#     yscale=:log,
    #yticks = 0:1:6, 
    yrange = (0,0.17),
#     yrange = (5e-6,5e1),
#      yrange = (0,0.4),
#     left_margin = 40px,
     right_margin = 20px,
#     bottom_margin = 10px,
#     top_margin = 10px
)


plot!(
    frame, 
    dfID.amp, 
    dfID.ratioPhi,
    label = L"$|\!H_{\phi} - H_{\phi,0}|\;/\;(|\!H_{\phi,0}| + |\!H_{\chi,0}|)$",
    linewidth=3, linecolor=phicolor
)

plot!(
    frame, 
    dfID.amp, 
    dfID.ratioChi,
    label = L"$|\!H_{\chi} - H_{\chi,0}|\;/\;(|\!H_{\phi,0}| + |\!H_{\chi,0}|)$",
    linewidth=3, linecolor=chicolor, linestyle=:dot
)



labelBoolean = false;

for seed in unique(df.stableRandomSeed)
    
    df_filtered = filter(row -> row.stableRandomSeed == seed, df)

    plot!(
        frame, 
        df_filtered.amp, 
        df_filtered.ratioPhi,
        label = if labelBoolean L"$|H_{\phi} - H_{\phi,0}|/|H_{0}|$" else false end,
        linewidth=0.5, linecolor=phicolor, alpha=0.1
    )

    plot!(
        frame, 
        df_filtered.amp, 
        df_filtered.ratioChi,
        label = if labelBoolean L"$|H_{\chi} - H_{\chi,0}|/|H_{0}|$" else false end,
        linewidth=0.5, linecolor=chicolor, linestyle=:dot, alpha=0.3
    )
    
    labelBoolean = false

end

# Plot mean line
plot!(
    frame, 
    stats_df.amp, 
    stats_df.mean_ratioPhi, 
    label = false, 
    linewidth=1, linecolor=phicolor
)
plot!(
    frame, 
    stats_df.amp, 
    stats_df.mean_ratioChi, 
    label = false, 
    linewidth=1, linecolor=chicolor, linestyle=:dot
)

# Shaded band ± std
plot!(
    frame, 
    stats_df.amp, 
    stats_df.mean_ratioPhi .+ stats_df.std_ratioPhi, 
    fillrange = stats_df.mean_ratioPhi .- stats_df.std_ratioPhi, 
    fillalpha=0.2, fillcolor=phicolor, linecolor=:false, label=false
)
plot!(
    frame, 
    stats_df.amp, 
    stats_df.mean_ratioChi .+ stats_df.std_ratioChi, 
    fillrange = stats_df.mean_ratioChi .- stats_df.std_ratioChi, 
    fillalpha=0.2, fillcolor=chicolor, linecolor=:false, label=false
)


plot_path = string("plots")

# create the directory if it does not yet exist
if !isdir(plot_path)
    print("Output plot directory does not exist. Creating it ...\n")
    mkdir(plot_path)
else
    print("Output plot directory already exists.\n")
end

savefig(
    frame, 
    joinpath(pwd(), plot_path, "extracted_component_energy.png")
)

Output plot directory already exists.


"/home/aaron/Dropbox/university/0_research/2022-2025/stable_ghosts_Paris/field_theory/calculations_aaron/julia_code/n+1_radial/v3/XX_THANOS/D=3_l=6_m=2_n=2/plots/extracted_component_energy.png"

In [19]:
frame = plot(
    layout=(1), size = (600,400), framestyle=:box, dpi = 400, 
    guidefont = "Computer Modern", tickfont = "Computer Modern",
    xguidefontsize = 16, yguidefontsize = 16, 
    legendfontsize = 16, titlefontsize = 16, tickfontsize = 16,
    legend=:topleft,
    title=L"$(N,\ell,m,n)=(3,6,2,2),\;\lambda_{mn}=1,\;\lambda_\ell=1$",
    xlabel=L"$\textrm{amplitude}\;\;A$",
    #xscale=:log,
    #xticks = 0:1:10,
    #xrange = (1.,13),
    ylabel=L"$\textrm{extracted}\;\textrm{energy}\;\textrm{fraction}$",
    yscale=:log,
    #yticks = 0:1:6, 
    #yrange = (8e-5,2e1),
    yrange = (5e-6,2e2),
#     left_margin = 40px,
     right_margin = 20px,
#     bottom_margin = 10px,
#     top_margin = 10px
)

plot!(
    frame, 
    dfID.amp, 
    2 * dfID.ratio,
#     label = L"$\sum_{i=\phi,\chi}|H_{i} - H_{i,0}|/(|H_{\phi,0}| + |H_{\chi,0}|)$",
    label = L"$\sum\!\;|\!H_{i} - H_{i,0}|\,/\;\sum\!\;|\!H_{i,0}|$",
    linewidth=3, linecolor=:black
)

labelBoolean = false;

for seed in unique(df.stableRandomSeed)
    
    df_filtered = filter(row -> row.stableRandomSeed == seed, df)

    plot!(
        frame, 
        df_filtered.amp, 
        2 * df_filtered.ratio,
        label = if labelBoolean L"$|H_{\phi} - H_{\phi,0}|/|H_{0}|$" else false end,
        linewidth=0.5, linecolor=:black
    )
    
    labelBoolean = false

end

plot_path = string("plots")

# create the directory if it does not yet exist
if !isdir(plot_path)
    print("Output plot directory does not exist. Creating it ...\n")
    mkdir(plot_path)
else
    print("Output plot directory already exists.\n")
end

savefig(
    frame, 
    joinpath(pwd(), plot_path, "extracted_component_energy_summed.png")
)

Output plot directory already exists.


"/home/aaron/Dropbox/university/0_research/2022-2025/stable_ghosts_Paris/field_theory/calculations_aaron/julia_code/n+1_radial/v3/XX_THANOS/D=3_l=6_m=2_n=2/plots/extracted_component_energy_summed.png"

# plot relative extracted kinetic energy